# Solar Energy Consumption and Production Analysis
This notebook analyzes solar energy consumption and production. It calculates various metrics, including PV production, unmet demand, and performance indicators, under different conditions (e.g., number of panels and batteries).



### Solar Energy Consumption and Production Analysis for ENSE3 Building (For one day)

In [ ]:
# --------------------------------
# Required Library Installation
# --------------------------------

import sys
import subprocess

# List of required Python libraries for the project
required_libraries = [
    'pandas',            # Data processing and manipulation
    'numpy',             # Numerical calculations
    'plotly',            # Interactive visualizations
    'ipywidgets',        # Interactive widgets for user input
    'matplotlib',        # Plotting capabilities 
]

# Function to check and install missing libraries
def install_missing_libraries(libraries):
    for lib in libraries:
        try:
            __import__(lib)
        except ImportError:
            print(f"Installing missing library: {lib}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
        else:
            print(f"✔ {lib} is already installed.")

# Run the function to install missing libraries
install_missing_libraries(required_libraries)


In [2]:
# main.ipynb
# ============================
# Solar + Wind Energy Consumption and Production Analysis for ENSE3 building
# ============================
#
# Author: Arbaz KHALID & Tshephang Komana
# Date: 2024-12-28
#
# Solar Energy Consumption and Production Analysis
# Enhanced to allow selecting a specific date within a chosen month.

## 1. Importing Modules and Setting Up Environment
import os
import sys
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import solar_analysis_script as sas

# Reload the updated solar_analysis_script module in case of changes
import importlib
importlib.reload(sas)

## 2. Define File Paths and Date Ranges
# Define file paths
consumption_file = 'Ense3buildingconsumption.csv'
pv_file = 'Ense3buildingPV.csv'

# Define separate date ranges
consumption_start_date = '2023-09-25 00:00:00'
consumption_end_date = '2024-09-25 00:00:00'

# Specify the actual column names in your consumption CSV
consumption_time_col = 'Time'
consumption_value_col = 'Consumption (kWh)'

# Number of solar panels
num_panels = 100  # Example value; adjust as needed

## 3. Load and Merge Data
try:
    merged_data = sas.load_data(
        consumption_file, 
        pv_file, 
        consumption_start_date, 
        consumption_end_date,
        consumption_sep=';', 
        consumption_time_col=consumption_time_col, 
        consumption_value_col=consumption_value_col,
        pv_sep=','
    )
    print("Data successfully loaded and merged after cleaning.")
except Exception as e:
    print(f"An error occurred while loading data: {e}")

# Ensure index is a DatetimeIndex
merged_data.index = pd.to_datetime(merged_data.index)

## 4. Define System Parameters
panel_capacity = 0.610  # kW per panel (610 W)
battery_capacity_per_unit = 57  # Example: 57 kWh per battery
converter_efficiency = 0.9    # 90% converter efficiency
initial_soc = 50              # 50% initial state of charge on 25 September 2023
battery_soc_tracking = {pd.Timestamp('2023-09-25'): initial_soc}  # Track SOC for each day

## 5. Create Interactive Widgets

# Widgets
num_panels_slider = widgets.IntSlider(value=0, min=0, max=1500, step=10, description='Num Panels:', continuous_update=False)
num_batteries_slider = widgets.IntSlider(value=0, min=0, max=100, step=1, description='Num Batteries:', continuous_update=False)
month_dropdown = widgets.Dropdown(options=[(pd.to_datetime(month, format='%m').strftime('%B'), month) for month in range(1, 13)], value=1, description='Month:')
date_dropdown = widgets.Dropdown(options=[], value=None, description='Date:')
message_output = widgets.Output()
plot_output = widgets.Output()

# Arrange widgets in a layout
widgets_box = widgets.VBox([widgets.HBox([num_panels_slider, num_batteries_slider]), widgets.HBox([month_dropdown, date_dropdown]), message_output])
display(widgets_box, plot_output)

## 6. Populate Date Dropdown Based on Selected Month

def populate_date_dropdown(change):
    selected_month = change['new']
    dates_in_month = merged_data[merged_data.index.month == selected_month].index.normalize().unique()
    date_options = [date.strftime('%Y-%m-%d') for date in dates_in_month]
    
    with message_output:
        message_output.clear_output()
        if date_options:
            date_dropdown.options = date_options
            date_dropdown.value = date_options[0]
            print(f"Available dates in {pd.to_datetime(selected_month, format='%m').strftime('%B')}:")
            print(date_options)
        else:
            date_dropdown.options = []
            date_dropdown.value = None
            print(f"No data available for {pd.to_datetime(selected_month, format='%m').strftime('%B')}.")

month_dropdown.observe(populate_date_dropdown, names='value')
populate_date_dropdown({'new': month_dropdown.value})

## 7. Define Function to Update the Graph Based on User Inputs

def update_graph(num_panels, num_batteries, selected_date):
    with plot_output:
        plot_output.clear_output()  # Clear previous plot
        
        if not selected_date:
            with message_output:
                message_output.clear_output()
                print("[Error] No date selected. Please select a valid date.")
            return
        
        # Convert selected_date to Timestamp
        selected_date = pd.Timestamp(selected_date)
        
        # Filter data for the selected date
        try:
            daily_data = merged_data.loc[selected_date.strftime('%Y-%m-%d')]
        except KeyError:
            with message_output:
                message_output.clear_output()
                print(f"[Error] No data available for the selected date: {selected_date}")
            return

        if daily_data.empty:
            with message_output:
                message_output.clear_output()
                print(f"[Warning] No data available for the selected date: {selected_date}")
            return

        # Ensure daily_data is a DataFrame
        if isinstance(daily_data, pd.Series):
            daily_data = daily_data.to_frame().T

        # Calculate PV production for the selected date and number of panels
        daily_pv_production = daily_data['PV_Production_kWh'].values * num_panels * panel_capacity

        # Retrieve SOC from the previous day
        previous_day = selected_date - pd.Timedelta(days=1)
        battery_soc = battery_soc_tracking.get(previous_day, initial_soc if previous_day >= pd.Timestamp('2023-09-25') else 0)

        # Initialize battery parameters
        battery_capacity = battery_capacity_per_unit * num_batteries
        energy_discharged = []
        energy_charged = []
        unmet_demand = []
        excess_energy = []

        for hour in range(24):
            consumption = daily_data.iloc[hour]['Consumption_kWh'] if hour in daily_data.index.hour else 0
            pv_production = daily_pv_production[hour] if hour < len(daily_pv_production) else 0

            if pv_production >= consumption:
                excess = pv_production - consumption
                if num_batteries > 0 and battery_soc < 100:
                    charge = min(excess * converter_efficiency, (100 - battery_soc) / 100 * battery_capacity)
                    energy_charged.append(charge)
                    battery_soc += (charge / battery_capacity) * 100
                    excess_energy.append(-(excess - charge))
                else:
                    energy_charged.append(0)
                    excess_energy.append(-excess)

                energy_discharged.append(0)
                unmet_demand.append(0)
            else:
                deficit = consumption - pv_production
                if num_batteries > 0 and 20 < battery_soc:
                    discharge = min(deficit, (battery_soc - 20) / 100 * battery_capacity)
                    energy_discharged.append(discharge)
                    battery_soc -= (discharge / battery_capacity) * 100
                    unmet_demand.append(deficit - discharge)
                else:
                    energy_discharged.append(0)
                    unmet_demand.append(deficit)
                energy_charged.append(0)
                excess_energy.append(0)

        # Update SOC tracking for this date
        battery_soc_tracking[selected_date] = battery_soc

        # Plot Solar PV Production, Unmet Demand, Energy Charged, Energy Discharged, and Electricity Demand
        fig = go.Figure()
        fig.add_trace(go.Bar(x=np.arange(24), y=daily_pv_production, name='Solar PV Production (kWh)', marker_color='orange'))
        fig.add_trace(go.Bar(x=np.arange(24), y=unmet_demand, name='Unmet Demand (kWh)', marker_color='red'))
        if num_batteries > 0:
            fig.add_trace(go.Bar(x=np.arange(24), y=[-val for val in energy_charged], name='Energy Charged to Battery (kWh)', marker_color='green'))
            fig.add_trace(go.Bar(x=np.arange(24), y=energy_discharged, name='Energy Discharged from Battery (kWh)', marker_color='purple'))
        fig.add_trace(go.Bar(x=np.arange(24), y=excess_energy, name='Excess Energy (kWh)', marker_color='cyan'))
        fig.add_trace(go.Scatter(x=np.arange(24), y=daily_data['Consumption_kWh'], mode='lines', name='Electricity Demand (kWh)', line=dict(color='blue', dash='dash')))

        fig.update_layout(
            title=f'Solar PV Production, Battery Management, and Electricity Demand on {selected_date.strftime("%Y-%m-%d")}',
            xaxis_title='Hour of the Day',
            yaxis_title='Energy (kWh)',
            xaxis=dict(tickvals=list(range(0, 24)), ticktext=[f'{h}:00' for h in range(0, 24)]),
            template='plotly_white',
            barmode='relative',
            height=600
        )

        fig.show()

out = widgets.interactive_output(
    update_graph,
    {'num_panels': num_panels_slider, 'num_batteries': num_batteries_slider, 'selected_date': date_dropdown}
)

display(out)


INFO:solar_analysis_script:Loading energy consumption data from file: Ense3buildingconsumption.csv


INFO:solar_analysis_script:Energy consumption data successfully loaded and filtered.
INFO:solar_analysis_script:Loading PV production data from file: Ense3buildingPV.csv
INFO:solar_analysis_script:'Time_End' column dropped from PV production data.
INFO:solar_analysis_script:PV production data successfully loaded, converted to kWh, and processed.
INFO:solar_analysis_script:Extending PV production data to cover 2023-09-25 00:00:00 to 2024-09-25 00:00:00.
INFO:solar_analysis_script:Original PV data duration: 364 days 23:00:00 (8760 hours)
INFO:solar_analysis_script:Total duration to cover: 366 days 00:00:00 (8784 hours)
INFO:solar_analysis_script:Repeating PV data 2 times to cover the desired period.
INFO:solar_analysis_script:PV production data extended from 2023-09-25 00:00:00 to 2024-09-24 23:00:00.
INFO:solar_analysis_script:Total PV production records after extension: 8784
INFO:solar_analysis_script:'Consumption (kWh)' column dropped from merged data.
INFO:solar_analysis_script:Consu

Data successfully loaded and merged after cleaning.


Output()

Output()